In [2]:
import json
from unskript import nbparams
from unskript.fwk.workflow import Task, Workflow
from unskript.secrets import ENV_MODE, ENV_MODE_LOCAL

env = {"ENV_MODE": "ENV_MODE_LOCAL"}
secret_store_cfg = {"SECRET_STORE_TYPE": "SECRET_STORE_TYPE_LOCAL"}

paramDict = {"inputRegion": "us-west-2"}
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
inputRegion = nbParamsObj.get('inputRegion')
w = Workflow(env, secret_store_cfg, None, global_vars=globals())

In [3]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint
from beartype import beartype

from beartype import beartype
@beartype
def aws_get_secrets_manager_secretARN_printer(output):
    if output is None:
        return
    pprint.pprint({"secret": output})


@beartype
@beartype
def aws_get_secrets_manager_secretARN(handle, region: str, secret_name:str) -> str:


    # Create a Secrets Manager client

    client = handle.client(
        service_name='secretsmanager',
        region_name=region
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e
    #print(get_secret_value_response)
    # Decrypts secret using the associated KMS key.
    secretArn = get_secret_value_response['ARN']
    return secretArn


task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "region": "",
    "secret_name": ,
    }''')
task.configure(outputName="secretArn")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_secrets_manager_secretARN, lego_printer=aws_get_secrets_manager_secretARN_printer, hdl=hdl, args=args)

In [24]:
import datetime

today = datetime.datetime.now()

yearmonth = today.strftime('%Y%m')
month = today.strftime('%m')
year =  today.strftime('%Y')
yearmonthday = yearmonth +"01"
#print("yearmonthday",yearmonthday)
if int(month) <12:
    nextMonth = int(month)+1
    if nextMonth < 10:
        nextMonthStr = "0" + str(nextMonth)
    else: 
        nextMonthStr = str(nextMonth)
if int(month) == 12:
    nextMonthStr = "01"
    year = year +1   
nextMonthYMD = year + nextMonthStr +"01"


tableName = 'awsbilling'+ yearmonth
dateRange = yearmonthday+'-'+nextMonthYMD
#print("dateRange", dateRange)

TruncateSQL = f"truncate table {tableName}"
print("TruncateSQL", TruncateSQL)
RebuildSql = f"copy {tableName} from 's3://unskript-billing-doug/all/unskript-billing-doug/{dateRange}/unskript-billing-doug-RedshiftManifest.json' credentials     'aws_iam_role=arn:aws:iam::100498623390:role/service-role/AmazonRedshift-CommandsAccessRole-20230103T181457' region 'us-west-2'    GZIP CSV IGNOREHEADER 1 TIMEFORMAT 'auto' manifest;"
print("RebuildSql", RebuildSql)



In [25]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype

from beartype import beartype

@beartype
def aws_create_redshift_query(handle, region: str,cluster:str, database:str, secretArn: str, query:str) -> str:

    # Input param validation.
    #major change
    client = handle.client('redshift-data', region_name=region)
    # define your query
    query = query
    #query = "SELECT * FROM PG_TABLE_DEF;"
    # execute the query
    response = client.execute_statement(
        ClusterIdentifier=cluster,
        Database=database,
        SecretArn=secretArn,
        Sql=query
    )
    resultId = response['Id']
    print(response)
    print("resultId",resultId)


    return resultId

#make a change
task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "region": "",
    "query": "",
    "cluster": """,
    "database": ,
    "secretArn": ""
    }''')
task.configure(outputName="truncateId")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_redshift_query,  hdl=hdl, args=args)

In [26]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
from unskript.connectors.aws import aws_get_paginator
import pprint
from beartype import beartype

from beartype import beartype
@beartype
def aws_get_redshift_query_results_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
@beartype
def aws_get_redshift_query_results(handle, region: str, queryId:str) -> Dict:

    client = handle.client('redshift-data', region_name=region)
    #result = aws_get_paginator(client, 
    #                           "get_statement_result", 
     #                           "TotalNumRows", 
     #                           Id='d7bda35c-7aa4-4414-9272-9e268320df40')
    response = client.describe_statement(
    Id=queryId
    )
    return response


task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "region": "",
    "queryId": ""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_redshift_query_results, lego_printer=aws_get_redshift_query_results_printer, hdl=hdl, args=args)

{'Instances': {'ClusterIdentifier': 'doug-billing-3',
               'CreatedAt': datetime.datetime(2023, 3, 10, 2, 4, 34, 181000, tzinfo=tzlocal()),
               'Duration': 47385559,
               'HasResultSet': False,
               'Id': 'eaf55bf3-f7bc-4106-b7bf-24c036de2c61',
               'QueryString': 'truncate table awsbilling202303',
               'RedshiftPid': 1073905767,
               'RedshiftQueryId': -1,
               'ResponseMetadata': {'HTTPHeaders': {'content-length': '421',
                                                    'content-type': 'application/x-amz-json-1.1',
                                                    'date': 'Fri, 10 Mar 2023 '
                                                            '02:04:37 GMT',
                                                    'x-amzn-requestid': '0700db68-0f75-41a3-8692-d1d91b907673'},
                                    'HTTPStatusCode': 200,
                                    'RequestId': '0700db68-0f75-41

In [27]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype

from beartype import beartype

@beartype
def aws_create_redshift_query(handle, region: str,cluster:str, database:str, secretArn: str, query:str) -> str:

    # Input param validation.
    #major change
    client = handle.client('redshift-data', region_name=region)
    # define your query
    query = query
    #query = "SELECT * FROM PG_TABLE_DEF;"
    # execute the query
    response = client.execute_statement(
        ClusterIdentifier=cluster,
        Database=database,
        SecretArn=secretArn,
        Sql=query
    )
    resultId = response['Id']
    print(response)
    print("resultId",resultId)


    return resultId

#make a change
task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "cluster": ,
    "database": ,
    "query": "",
    "region": "",
    "secretArn": ""
    }''')
task.configure(outputName="redshiftUpdateId")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_redshift_query,  hdl=hdl, args=args)

In [29]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
from unskript.connectors.aws import aws_get_paginator
import pprint
from beartype import beartype

from beartype import beartype
@beartype
def aws_get_redshift_query_results_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
@beartype
def aws_get_redshift_query_results(handle, region: str, queryId:str) -> Dict:

    client = handle.client('redshift-data', region_name=region)
    #result = aws_get_paginator(client, 
    #                           "get_statement_result", 
     #                           "TotalNumRows", 
     #                           Id='d7bda35c-7aa4-4414-9272-9e268320df40')
    response = client.describe_statement(
    Id=queryId
    )
    return response


task = Task(Workflow())
task.configure(credentialsJson='''{}''')
task.configure(inputParamsJson='''{
    "region": "inputRegion",
    "queryId": "redshiftUpdateId"
    }''')


task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_redshift_query_results, lego_printer=aws_get_redshift_query_results_printer, hdl=hdl, args=args)